# 5. Классические методы сглаживания временных рядов

## Введение

Методы сглаживания (smoothing) — это класс техник для выделения сигнала из зашумленных данных, устранения краткосрочных колебаний и выявления долгосрочных трендов. Эти методы широко применяются как для исследовательского анализа данных, так и для краткосрочного прогнозирования.

В этом ноутбуке мы рассмотрим:
- Скользящие средние (Moving Averages)
- Простое экспоненциальное сглаживание (Simple Exponential Smoothing, SES)
- Метод Хольта для данных с трендом (Holt's Linear Trend)
- Метод Хольта-Винтерса для данных с трендом и сезонностью (Holt-Winters)
- Практическое применение и сравнение методов

## 1. Теоретические основы

### 1.1. Скользящие средние (Moving Averages)

**Простое скользящее среднее (Simple Moving Average, SMA)**

Скользящее среднее порядка $m$ вычисляется как:

$$\hat{y}_t = \frac{1}{m}\sum_{i=0}^{m-1} y_{t-i}$$

где $m$ — размер окна.

**Свойства:**
- Простота реализации и интерпретации
- Все наблюдения в окне имеют одинаковый вес
- Задержка (lag) в $\frac{m}{2}$ периодов
- Сглаживает случайные колебания, но теряет $m-1$ первых наблюдений

**Взвешенное скользящее среднее (Weighted Moving Average, WMA)**

$$\hat{y}_t = \sum_{i=0}^{m-1} w_i y_{t-i}$$

где $w_i$ — веса, такие что $\sum w_i = 1$. Обычно более свежим наблюдениям присваиваются большие веса.

**Центрированное скользящее среднее (Centered Moving Average, CMA)**

Для нечетного $m = 2k+1$:

$$\hat{y}_t = \frac{1}{m}\sum_{i=-k}^{k} y_{t+i}$$

Используется для декомпозиции временных рядов, так как не вносит задержки.

### 1.2. Экспоненциальное сглаживание (Exponential Smoothing)

**Простое экспоненциальное сглаживание (SES)**

Прогноз вычисляется как взвешенная комбинация текущего наблюдения и предыдущего прогноза:

$$\hat{y}_{t+1} = \alpha y_t + (1-\alpha)\hat{y}_t$$

или эквивалентно:

$$\hat{y}_{t+1} = \hat{y}_t + \alpha(y_t - \hat{y}_t)$$

где:
- $\alpha \in [0, 1]$ — параметр сглаживания
- $y_t$ — фактическое значение в момент $t$
- $\hat{y}_t$ — прогноз для момента $t$

**Свойства:**
- Экспоненциально убывающие веса для прошлых наблюдений
- Большое $\alpha$ (близко к 1) → быстрая адаптация к изменениям
- Малое $\alpha$ (близко к 0) → сильное сглаживание
- Подходит для рядов без тренда и сезонности

**Раскрытая форма:**

$$\hat{y}_{t+1} = \alpha y_t + \alpha(1-\alpha)y_{t-1} + \alpha(1-\alpha)^2 y_{t-2} + ...$$

### 1.3. Метод Хольта (Holt's Linear Trend Method)

Расширяет SES для рядов с трендом, вводя две компоненты: уровень и тренд.

**Уравнения:**

1. Уравнение уровня:
$$\ell_t = \alpha y_t + (1-\alpha)(\ell_{t-1} + b_{t-1})$$

2. Уравнение тренда:
$$b_t = \beta(\ell_t - \ell_{t-1}) + (1-\beta)b_{t-1}$$

3. Прогноз на $h$ шагов вперед:
$$\hat{y}_{t+h|t} = \ell_t + h \cdot b_t$$

где:
- $\ell_t$ — оценка уровня ряда в момент $t$
- $b_t$ — оценка тренда в момент $t$
- $\alpha$ — параметр сглаживания уровня
- $\beta$ — параметр сглаживания тренда

**Демпфированный тренд (Damped Trend)**

Чтобы избежать бесконечного роста прогнозов, можно использовать демпфированный тренд:

$$\hat{y}_{t+h|t} = \ell_t + (\phi + \phi^2 + ... + \phi^h) b_t$$

где $\phi \in (0, 1)$ — параметр демпфирования.

### 1.4. Метод Хольта-Винтерса (Holt-Winters)

Расширяет метод Хольта для рядов с сезонностью. Существует две версии: аддитивная и мультипликативная.

**Аддитивная модель Хольта-Винтерса** (когда сезонные колебания примерно постоянны):

1. Уравнение уровня:
$$\ell_t = \alpha(y_t - s_{t-m}) + (1-\alpha)(\ell_{t-1} + b_{t-1})$$

2. Уравнение тренда:
$$b_t = \beta(\ell_t - \ell_{t-1}) + (1-\beta)b_{t-1}$$

3. Уравнение сезонности:
$$s_t = \gamma(y_t - \ell_{t-1} - b_{t-1}) + (1-\gamma)s_{t-m}$$

4. Прогноз:
$$\hat{y}_{t+h|t} = \ell_t + h \cdot b_t + s_{t+h-m(k+1)}$$

где $k = \lfloor(h-1)/m\rfloor$ и $m$ — длина сезона.

**Мультипликативная модель Хольта-Винтерса** (когда сезонные колебания пропорциональны уровню):

1. Уравнение уровня:
$$\ell_t = \alpha\frac{y_t}{s_{t-m}} + (1-\alpha)(\ell_{t-1} + b_{t-1})$$

2. Уравнение тренда:
$$b_t = \beta(\ell_t - \ell_{t-1}) + (1-\beta)b_{t-1}$$

3. Уравнение сезонности:
$$s_t = \gamma\frac{y_t}{\ell_{t-1} + b_{t-1}} + (1-\gamma)s_{t-m}$$

4. Прогноз:
$$\hat{y}_{t+h|t} = (\ell_t + h \cdot b_t) \cdot s_{t+h-m(k+1)}$$

**Параметры:**
- $\alpha$ — сглаживание уровня
- $\beta$ — сглаживание тренда
- $\gamma$ — сглаживание сезонности
- Все параметры в диапазоне $[0, 1]$

## 2. Практическая реализация

### 2.1. Подготовка окружения

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)

print("Библиотеки загружены успешно!")

### 2.2. Генерация синтетических данных

In [ ]:
np.random.seed(42)

# Временная шкала
n = 200
time = np.arange(n)

# 1. Ряд без тренда и сезонности (для SES)
level = 50
noise = np.random.randn(n) * 5
simple_series = level + noise

# 2. Ряд с трендом (для метода Хольта)
trend = 0.3 * time
trend_series = level + trend + noise

# 3. Ряд с трендом и сезонностью (для Хольта-Винтерса)
seasonal_period = 12
seasonality = 10 * np.sin(2 * np.pi * time / seasonal_period)
seasonal_series = level + 0.2 * time + seasonality + noise * 0.5

# Создание DataFrame
df = pd.DataFrame({
    'time': time,
    'simple': simple_series,
    'trend': trend_series,
    'seasonal': seasonal_series
})

# Визуализация
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

axes[0].plot(df['time'], df['simple'], label='Без тренда и сезонности', alpha=0.7)
axes[0].set_title('Ряд без тренда и сезонности')
axes[0].set_ylabel('Значение')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(df['time'], df['trend'], label='С трендом', color='C1', alpha=0.7)
axes[1].set_title('Ряд с трендом')
axes[1].set_ylabel('Значение')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(df['time'], df['seasonal'], label='С трендом и сезонностью', color='C2', alpha=0.7)
axes[2].set_title('Ряд с трендом и сезонностью')
axes[2].set_xlabel('Время')
axes[2].set_ylabel('Значение')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Скользящие средние

### 3.1. Простое скользящее среднее

In [ ]:
# Различные размеры окна
windows = [5, 10, 20]

fig, axes = plt.subplots(len(windows), 1, figsize=(14, 12))

for idx, window in enumerate(windows):
    sma = df['seasonal'].rolling(window=window).mean()
    
    axes[idx].plot(df['time'], df['seasonal'], label='Исходный ряд', alpha=0.5, linewidth=1)
    axes[idx].plot(df['time'], sma, label=f'SMA (окно={window})', linewidth=2)
    axes[idx].set_title(f'Простое скользящее среднее с окном {window}')
    axes[idx].set_ylabel('Значение')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

axes[-1].set_xlabel('Время')
plt.tight_layout()
plt.show()

print("\n📊 Наблюдения:")
print("- Маленькое окно (5): Следует за данными, но все еще шумное")
print("- Среднее окно (10): Баланс между сглаживанием и реакцией")
print("- Большое окно (20): Сильное сглаживание, большая задержка")

### 3.2. Центрированное скользящее среднее

In [ ]:
window = 12  # период сезонности

# Обычное скользящее среднее
sma = df['seasonal'].rolling(window=window).mean()

# Центрированное скользящее среднее
cma = df['seasonal'].rolling(window=window, center=True).mean()

plt.figure(figsize=(14, 6))
plt.plot(df['time'], df['seasonal'], label='Исходный ряд', alpha=0.5, linewidth=1)
plt.plot(df['time'], sma, label=f'Обычное SMA (окно={window})', linewidth=2)
plt.plot(df['time'], cma, label=f'Центрированное SMA (окно={window})', linewidth=2, linestyle='--')
plt.title('Сравнение обычного и центрированного скользящего среднего')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Центрированное скользящее среднее не имеет временной задержки")
print("   и лучше подходит для декомпозиции, но теряет данные с обоих концов.")

## 4. Простое экспоненциальное сглаживание (SES)

### 4.1. Реализация с разными параметрами α

In [ ]:
# Разделение на train/test
train_size = int(len(df) * 0.8)
train_simple = df['simple'][:train_size]
test_simple = df['simple'][train_size:]

# Различные значения alpha
alphas = [0.1, 0.3, 0.7, 0.9]
results_ses = {}

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.ravel()

for idx, alpha in enumerate(alphas):
    # Обучение модели
    model = SimpleExpSmoothing(train_simple).fit(smoothing_level=alpha, optimized=False)
    
    # Прогноз
    fitted = model.fittedvalues
    forecast = model.forecast(steps=len(test_simple))
    
    # Метрики
    rmse = np.sqrt(mean_squared_error(test_simple, forecast))
    mae = mean_absolute_error(test_simple, forecast)
    
    results_ses[alpha] = {'RMSE': rmse, 'MAE': mae}
    
    # Визуализация
    axes[idx].plot(train_simple.index, train_simple, label='Train', alpha=0.7)
    axes[idx].plot(test_simple.index, test_simple, label='Test', alpha=0.7)
    axes[idx].plot(train_simple.index, fitted, label='Fitted', linewidth=2)
    axes[idx].plot(test_simple.index, forecast, label='Forecast', linewidth=2, linestyle='--')
    axes[idx].axvline(train_size, color='black', linestyle=':', alpha=0.5)
    axes[idx].set_title(f'SES: α={alpha}\nRMSE={rmse:.2f}, MAE={mae:.2f}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Сравнение результатов
print("\n" + "="*60)
print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ SES")
print("="*60)
results_df = pd.DataFrame(results_ses).T
print(results_df)
print("\n💡 Меньшее α → сильнее сглаживание, больше α → быстрее реакция")

### 4.2. Автоматический подбор параметров

In [ ]:
# Автоматическая оптимизация параметра alpha
model_auto = SimpleExpSmoothing(train_simple).fit(optimized=True)

print(f"\n🎯 Оптимальное значение α: {model_auto.params['smoothing_level']:.4f}")
print(f"AIC: {model_auto.aic:.2f}")
print(f"BIC: {model_auto.bic:.2f}")

# Прогноз
forecast_auto = model_auto.forecast(steps=len(test_simple))
rmse_auto = np.sqrt(mean_squared_error(test_simple, forecast_auto))

plt.figure(figsize=(14, 6))
plt.plot(train_simple.index, train_simple, label='Train', alpha=0.7)
plt.plot(test_simple.index, test_simple, label='Test', alpha=0.7)
plt.plot(train_simple.index, model_auto.fittedvalues, label='Fitted', linewidth=2)
plt.plot(test_simple.index, forecast_auto, label='Forecast', linewidth=2, linestyle='--')
plt.axvline(train_size, color='black', linestyle=':', alpha=0.5)
plt.title(f'SES с автоматически подобранным α={model_auto.params["smoothing_level"]:.4f}\nRMSE={rmse_auto:.2f}')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Метод Хольта (для рядов с трендом)

### 5.1. Базовая реализация

In [ ]:
# Разделение данных с трендом
train_trend = df['trend'][:train_size]
test_trend = df['trend'][train_size:]

# Метод Хольта
model_holt = ExponentialSmoothing(
    train_trend,
    trend='add',  # аддитивный тренд
    seasonal=None
).fit(optimized=True)

print("\n" + "="*60)
print("ПАРАМЕТРЫ МЕТОДА ХОЛЬТА")
print("="*60)
print(f"α (сглаживание уровня): {model_holt.params['smoothing_level']:.4f}")
print(f"β (сглаживание тренда): {model_holt.params['smoothing_trend']:.4f}")
print(f"AIC: {model_holt.aic:.2f}")
print(f"BIC: {model_holt.bic:.2f}")

# Прогноз
forecast_holt = model_holt.forecast(steps=len(test_trend))
rmse_holt = np.sqrt(mean_squared_error(test_trend, forecast_holt))

plt.figure(figsize=(14, 6))
plt.plot(train_trend.index, train_trend, label='Train', alpha=0.7)
plt.plot(test_trend.index, test_trend, label='Test', alpha=0.7)
plt.plot(train_trend.index, model_holt.fittedvalues, label='Fitted', linewidth=2)
plt.plot(test_trend.index, forecast_holt, label='Forecast', linewidth=2, linestyle='--')
plt.axvline(train_size, color='black', linestyle=':', alpha=0.5)
plt.title(f'Метод Хольта (линейный тренд)\nRMSE={rmse_holt:.2f}')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 5.2. Сравнение с SES

In [ ]:
# SES на данных с трендом (для сравнения)
model_ses_trend = SimpleExpSmoothing(train_trend).fit(optimized=True)
forecast_ses_trend = model_ses_trend.forecast(steps=len(test_trend))
rmse_ses_trend = np.sqrt(mean_squared_error(test_trend, forecast_ses_trend))

# Визуализация сравнения
plt.figure(figsize=(14, 6))
plt.plot(train_trend.index, train_trend, label='Train', alpha=0.5)
plt.plot(test_trend.index, test_trend, label='Test', color='black', alpha=0.7)
plt.plot(test_trend.index, forecast_ses_trend, label=f'SES (RMSE={rmse_ses_trend:.2f})', linewidth=2, linestyle='--')
plt.plot(test_trend.index, forecast_holt, label=f'Holt (RMSE={rmse_holt:.2f})', linewidth=2, linestyle='--')
plt.axvline(train_size, color='black', linestyle=':', alpha=0.5)
plt.title('Сравнение SES и метода Хольта на данных с трендом')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n📈 Метод Хольта значительно лучше учитывает тренд!")

### 5.3. Демпфированный тренд

In [ ]:
# Метод Хольта с демпфированным трендом
model_holt_damped = ExponentialSmoothing(
    train_trend,
    trend='add',
    damped_trend=True,
    seasonal=None
).fit(optimized=True)

print(f"\nПараметр демпфирования φ: {model_holt_damped.params['damping_trend']:.4f}")

# Прогноз на большее количество шагов для демонстрации эффекта
steps_ahead = 50
forecast_holt_long = model_holt.forecast(steps=steps_ahead)
forecast_damped_long = model_holt_damped.forecast(steps=steps_ahead)

plt.figure(figsize=(14, 6))
plt.plot(train_trend.index, train_trend, label='Train', alpha=0.7)
future_index = np.arange(train_size, train_size + steps_ahead)
plt.plot(future_index, forecast_holt_long, label='Holt (обычный)', linewidth=2, linestyle='--')
plt.plot(future_index, forecast_damped_long, label='Holt (демпфированный)', linewidth=2, linestyle='--')
plt.axvline(train_size, color='black', linestyle=':', alpha=0.5)
plt.title('Сравнение обычного и демпфированного тренда (длинный прогноз)')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Демпфированный тренд предотвращает бесконечный рост прогнозов")
print("   и часто дает более реалистичные долгосрочные прогнозы.")

## 6. Метод Хольта-Винтерса (для рядов с трендом и сезонностью)

### 6.1. Аддитивная модель

In [ ]:
# Разделение данных с сезонностью
train_seasonal = df['seasonal'][:train_size]
test_seasonal = df['seasonal'][train_size:]

# Метод Хольта-Винтерса (аддитивная модель)
model_hw_add = ExponentialSmoothing(
    train_seasonal,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit(optimized=True)

print("\n" + "="*60)
print("ПАРАМЕТРЫ ХОЛЬТА-ВИНТЕРСА (АДДИТИВНАЯ МОДЕЛЬ)")
print("="*60)
print(f"α (сглаживание уровня): {model_hw_add.params['smoothing_level']:.4f}")
print(f"β (сглаживание тренда): {model_hw_add.params['smoothing_trend']:.4f}")
print(f"γ (сглаживание сезонности): {model_hw_add.params['smoothing_seasonal']:.4f}")
print(f"AIC: {model_hw_add.aic:.2f}")
print(f"BIC: {model_hw_add.bic:.2f}")

# Прогноз
forecast_hw_add = model_hw_add.forecast(steps=len(test_seasonal))
rmse_hw_add = np.sqrt(mean_squared_error(test_seasonal, forecast_hw_add))
mae_hw_add = mean_absolute_error(test_seasonal, forecast_hw_add)

plt.figure(figsize=(14, 6))
plt.plot(train_seasonal.index, train_seasonal, label='Train', alpha=0.7)
plt.plot(test_seasonal.index, test_seasonal, label='Test', alpha=0.7)
plt.plot(train_seasonal.index, model_hw_add.fittedvalues, label='Fitted', linewidth=2)
plt.plot(test_seasonal.index, forecast_hw_add, label='Forecast', linewidth=2, linestyle='--')
plt.axvline(train_size, color='black', linestyle=':', alpha=0.5)
plt.title(f'Хольт-Винтерс (аддитивная модель)\nRMSE={rmse_hw_add:.2f}, MAE={mae_hw_add:.2f}')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 6.2. Мультипликативная модель

In [ ]:
# Генерация ряда с мультипликативной сезонностью
np.random.seed(42)
multiplicative_series = (50 + 0.3 * time) * (1 + 0.2 * np.sin(2 * np.pi * time / 12)) + np.random.randn(n) * 2
df['multiplicative'] = multiplicative_series

train_mult = df['multiplicative'][:train_size]
test_mult = df['multiplicative'][train_size:]

# Хольт-Винтерс с мультипликативной сезонностью
model_hw_mult = ExponentialSmoothing(
    train_mult,
    trend='add',
    seasonal='mul',
    seasonal_periods=12
).fit(optimized=True)

forecast_hw_mult = model_hw_mult.forecast(steps=len(test_mult))
rmse_hw_mult = np.sqrt(mean_squared_error(test_mult, forecast_hw_mult))

# Сравнение аддитивной и мультипликативной моделей
model_hw_add_mult = ExponentialSmoothing(
    train_mult,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit(optimized=True)
forecast_hw_add_mult = model_hw_add_mult.forecast(steps=len(test_mult))
rmse_hw_add_mult = np.sqrt(mean_squared_error(test_mult, forecast_hw_add_mult))

plt.figure(figsize=(14, 6))
plt.plot(train_mult.index, train_mult, label='Train', alpha=0.5)
plt.plot(test_mult.index, test_mult, label='Test', color='black', alpha=0.7)
plt.plot(test_mult.index, forecast_hw_add_mult, 
         label=f'Аддитивная (RMSE={rmse_hw_add_mult:.2f})', linewidth=2, linestyle='--')
plt.plot(test_mult.index, forecast_hw_mult, 
         label=f'Мультипликативная (RMSE={rmse_hw_mult:.2f})', linewidth=2, linestyle='--')
plt.axvline(train_size, color='black', linestyle=':', alpha=0.5)
plt.title('Сравнение аддитивной и мультипликативной моделей Хольта-Винтерса')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 Выбирайте:")
print("   - Аддитивную модель: когда сезонные колебания примерно постоянны")
print("   - Мультипликативную модель: когда амплитуда сезонности растет с уровнем ряда")

## 7. Сравнение всех методов

In [ ]:
# Применение всех методов к сезонному ряду
models = {
    'SES': SimpleExpSmoothing(train_seasonal).fit(optimized=True),
    'Holt': ExponentialSmoothing(train_seasonal, trend='add', seasonal=None).fit(optimized=True),
    'HW (add)': ExponentialSmoothing(train_seasonal, trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True),
}

# Сравнение прогнозов
results_comparison = {}

plt.figure(figsize=(14, 8))
plt.plot(train_seasonal.index, train_seasonal, label='Train', alpha=0.5, linewidth=1)
plt.plot(test_seasonal.index, test_seasonal, label='Test (факт)', color='black', linewidth=2, alpha=0.7)

for name, model in models.items():
    forecast = model.forecast(steps=len(test_seasonal))
    rmse = np.sqrt(mean_squared_error(test_seasonal, forecast))
    mae = mean_absolute_error(test_seasonal, forecast))
    
    results_comparison[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'AIC': model.aic,
        'BIC': model.bic
    }
    
    plt.plot(test_seasonal.index, forecast, label=f'{name} (RMSE={rmse:.2f})', linewidth=2, linestyle='--')

plt.axvline(train_size, color='black', linestyle=':', alpha=0.5)
plt.title('Сравнение всех методов сглаживания')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Таблица результатов
print("\n" + "="*80)
print("СРАВНЕНИЕ МЕТОДОВ")
print("="*80)
comparison_df = pd.DataFrame(results_comparison).T
comparison_df = comparison_df.round(2)
print(comparison_df)
print("\n🏆 Лучший метод по RMSE:", comparison_df['RMSE'].idxmin())

## 8. Практические рекомендации

### 8.1. Выбор метода

| Характеристики ряда | Рекомендуемый метод |
|---------------------|---------------------|
| Без тренда и сезонности | SES или SMA |
| С трендом, без сезонности | Holt (линейный или демпфированный) |
| С сезонностью (постоянной амплитуды) | Holt-Winters (аддитивная) |
| С сезонностью (растущей амплитуды) | Holt-Winters (мультипликативная) |
| Сложная нелинейность | Рассмотреть ARIMA, ML-методы |

### 8.2. Настройка параметров

**Параметр сглаживания α:**
- α близко к 1 (0.7-0.9): быстрая реакция на изменения, минимальное сглаживание
- α близко к 0 (0.1-0.3): медленная реакция, сильное сглаживание
- Для стабильных рядов: используйте меньшее α
- Для волатильных рядов: используйте большее α

**Автоматический подбор:**
- Используйте `optimized=True` для автоматической оптимизации
- Параметры подбираются по минимуму суммы квадратов ошибок
- Проверяйте адекватность на валидационной выборке

### 8.3. Оценка качества

**Метрики:**
- RMSE (Root Mean Squared Error): штрафует большие ошибки
- MAE (Mean Absolute Error): средняя абсолютная ошибка
- MAPE (Mean Absolute Percentage Error): относительная ошибка
- AIC/BIC: для сравнения моделей (меньше - лучше)

**Диагностика:**
- Анализ остатков (должны быть случайными)
- ACF остатков (не должно быть автокорреляции)
- Визуальное сравнение прогноза с фактом

### 8.4. Преимущества и недостатки

**Преимущества:**
- ✅ Простота и интерпретируемость
- ✅ Малое количество параметров
- ✅ Быстрые вычисления
- ✅ Хорошо работают для краткосрочных прогнозов
- ✅ Подходят для онлайн-обучения (инкрементальное обновление)

**Недостатки:**
- ❌ Ограниченная гибкость
- ❌ Предполагают простую структуру ряда
- ❌ Не подходят для сложных нелинейных зависимостей
- ❌ Качество ухудшается для долгосрочных прогнозов
- ❌ Чувствительны к выбросам

## 9. Задания для самостоятельной работы

### Задание 1: Базовый уровень

Загрузите реальный временной ряд (например, ежедневные продажи, температуру или цены) и:

1. Визуализируйте ряд и определите наличие тренда и сезонности
2. Разделите данные на train/test (80%/20%)
3. Примените SES с различными значениями α (0.1, 0.3, 0.5, 0.7, 0.9)
4. Сравните результаты по RMSE и MAE
5. Визуализируйте прогнозы всех моделей
6. Выберите оптимальное значение α

In [ ]:
# TODO: Задание 1
# Загрузите данные и примените SES с разными параметрами

pass

### Задание 2: Продвинутый уровень

Для ряда с трендом и сезонностью:

1. Примените все изученные методы: SMA, SES, Holt, Holt-Winters (аддитивная и мультипликативная)
2. Используйте автоматический подбор параметров для каждого метода
3. Создайте сводную таблицу с метриками: RMSE, MAE, MAPE, AIC, BIC
4. Постройте графики прогнозов всех методов на одной фигуре
5. Проведите анализ остатков для лучшей модели:
   - График остатков во времени
   - Гистограмма остатков
   - ACF остатков
   - QQ-plot
6. Сделайте выводы о том, какой метод лучше подходит для ваших данных и почему

In [ ]:
# TODO: Задание 2
# Комплексное сравнение методов

pass

### Задание 3: Исследовательский уровень

Проведите исследование эффективности методов сглаживания:

1. Соберите 5-7 различных временных рядов из разных областей
2. Для каждого ряда:
   - Определите характеристики (тренд, сезонность, волатильность)
   - Примените все методы сглаживания
   - Проведите кросс-валидацию с различными размерами обучающей выборки
3. Создайте итоговую таблицу с результатами для всех рядов
4. Постройте визуализации:
   - Heatmap с метриками качества для каждой комбинации (ряд × метод)
   - Boxplot распределения ошибок по методам
   - Scatter plot: RMSE vs вычислительное время
5. Ответьте на вопросы:
   - Какой метод наиболее универсален?
   - Для каких типов данных какой метод лучше?
   - Как связаны характеристики ряда и эффективность методов?
   - Есть ли компромисс между точностью и вычислительной сложностью?

**Дополнительно (необязательно):**
- Реализуйте собственный класс для автоматического выбора лучшего метода
- Исследуйте влияние выбросов на качество прогнозов
- Сравните методы сглаживания с ARIMA-моделями

In [ ]:
# TODO: Задание 3
# Комплексное исследование

pass

## 10. Контрольные вопросы

1. В чем разница между простым и взвешенным скользящим средним?
2. Почему центрированное скользящее среднее лучше подходит для декомпозиции?
3. Как выбрать размер окна для скользящего среднего?
4. Объясните, как работает простое экспоненциальное сглаживание (SES).
5. Что означает параметр α в SES и как его выбирать?
6. В чем преимущество экспоненциального сглаживания перед скользящим средним?
7. Когда следует использовать метод Хольта вместо SES?
8. Что такое демпфированный тренд и зачем он нужен?
9. В чем разница между аддитивной и мультипликативной моделями Хольта-Винтерса?
10. Как определить, какую модель (аддитивную или мультипликативную) использовать?
11. Почему методы сглаживания лучше работают для краткосрочных прогнозов?
12. Какие ограничения имеют методы экспоненциального сглаживания?

## 11. Дополнительные материалы

**Литература:**
- Hyndman, R.J., Athanasopoulos, G. (2021). *Forecasting: Principles and Practice* (3rd ed). OTexts.
  - Глава 8: Exponential smoothing
  - https://otexts.com/fpp3/expsmooth.html
- Gardner, E.S. (2006). "Exponential smoothing: The state of the art—Part II". *International journal of forecasting*, 22(4), 637-666.
- Makridakis, S., Wheelwright, S.C., Hyndman, R.J. (1998). *Forecasting: Methods and Applications*. John Wiley & Sons.

**Онлайн-ресурсы:**
- Statsmodels Documentation: Exponential Smoothing
  - https://www.statsmodels.org/stable/examples/notebooks/generated/exponential_smoothing.html
- "Exponential smoothing" на сайте Duke University
  - https://people.duke.edu/~rnau/411avg.htm
- Rob Hyndman's blog: "Forecasting with exponential smoothing"
  - https://robjhyndman.com/

**Практические инструменты:**
- `statsmodels.tsa.holtwinters`: реализация в Python
- `forecast` package в R: более продвинутая реализация
- `sktime`: scikit-learn compatible библиотека для временных рядов